In [1]:
from pathlib import Path
import numpy as np
import heapq
from collections import defaultdict
import seaborn as sns
import pandas as pd
import matplotlib.pylab as plt
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from itertools import chain
from sklearn.cluster import KMeans
from sklearn.preprocessing import OneHotEncoder
from sklearn import preprocessing
from sklearn import metrics

In [2]:
#We read the dataset given.

In [3]:
fp = pd.read_csv('C:/Users/User/StudentClass.csv')
fp

,StudentID,GRADE,ClassWeekDay,ClassMeetTime,ClassDate,ClassTitle
0,1.0,A,MW,15:30:00,9/1/2004,Math
1,1.0,A,M,19:00:00,9/1/2004,English
2,1.0,C,W,19:00:00,9/1/2004,Physics
3,1.0,B,MW,11:00:00,9/1/2004,Art
4,1.0,B,MW,12:30:00,9/1/2005,SocialStudy
...,...,...,...,...,...,...
18705,NaN,NaN,NaN,NaN,NaN,NaN
18706,NaN,NaN,NaN,NaN,NaN,NaN
18707,NaN,NaN,NaN,NaN,NaN,NaN
18708,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
fp=fp.dropna()

In [5]:
fp

,StudentID,GRADE,ClassWeekDay,ClassMeetTime,ClassDate,ClassTitle
0,1.0,A,MW,15:30:00,9/1/2004,Math
1,1.0,A,M,19:00:00,9/1/2004,English
2,1.0,C,W,19:00:00,9/1/2004,Physics
3,1.0,B,MW,11:00:00,9/1/2004,Art
4,1.0,B,MW,12:30:00,9/1/2005,SocialStudy
...,...,...,...,...,...,...
9995,1000.0,C,MW,15:30:00,9/1/2008,Physics
9996,1000.0,C,MW,14:00:00,9/1/2008,Math
9997,1000.0,C,TTH,12:30:00,9/1/2008,ForeignLanguage
9998,1000.0,B,TU,19:00:00,9/1/2009,Music


In [6]:
## Converting the studentid into integer datatype. 

In [7]:
fp['StudentID'] = fp['StudentID'].astype(int)
fp

C:\Users\User\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,StudentID,GRADE,ClassWeekDay,ClassMeetTime,ClassDate,ClassTitle
0,1,A,MW,15:30:00,9/1/2004,Math
1,1,A,M,19:00:00,9/1/2004,English
2,1,C,W,19:00:00,9/1/2004,Physics
3,1,B,MW,11:00:00,9/1/2004,Art
4,1,B,MW,12:30:00,9/1/2005,SocialStudy
...,...,...,...,...,...,...
9995,1000,C,MW,15:30:00,9/1/2008,Physics
9996,1000,C,MW,14:00:00,9/1/2008,Math
9997,1000,C,TTH,12:30:00,9/1/2008,ForeignLanguage
9998,1000,B,TU,19:00:00,9/1/2009,Music


In [8]:
fp_df = pd.read_csv('StudentGraduate.csv')
fp_df

,ID,HS_ENGLISH,HS_MATH,HS_HISTORY,HS_LAB_SCIENCES,HS_FOREIGN_LANGUAGE,HS_ART,HS_ELECTIVES,HS_GPA,sex,WRITINGScore,GraduateONTime,Major
0,1,8,8,7,8,6,4,1,3.691959,1,428,0,FIN
1,2,8,10,7,6,8,10,0,3.712993,1,558,0,MGMT
2,3,8,7,6,6,4,4,0,4.373719,1,634,1,MKTG
3,4,8,8,6,6,4,2,0,4.046570,0,386,0,MKTG
4,5,8,6,6,4,4,3,1,2.560076,0,381,0,MGMT
...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,996,8,8,7,6,6,3,2,3.002898,1,441,0,MGMT
996,997,8,10,6,7,6,2,0,3.910183,0,443,1,ACCT
997,998,8,10,6,8,6,6,0,3.267001,0,515,0,MGMT
998,999,8,8,7,8,8,2,1,3.474561,0,459,0,MKTG


In [9]:
# Merging two dataset 'StudentGraduate' and 'StudentClass' to built a classification model.

In [10]:
fp = pd.merge(fp, fp_df, left_on='StudentID', right_on='ID', how='left')
fp

,StudentID,GRADE,ClassWeekDay,ClassMeetTime,ClassDate,ClassTitle,ID,HS_ENGLISH,HS_MATH,HS_HISTORY,HS_LAB_SCIENCES,HS_FOREIGN_LANGUAGE,HS_ART,HS_ELECTIVES,HS_GPA,sex,WRITINGScore,GraduateONTime,Major
0,1,A,MW,15:30:00,9/1/2004,Math,1,8,8,7,8,6,4,1,3.691959,1,428,0,FIN
1,1,A,M,19:00:00,9/1/2004,English,1,8,8,7,8,6,4,1,3.691959,1,428,0,FIN
2,1,C,W,19:00:00,9/1/2004,Physics,1,8,8,7,8,6,4,1,3.691959,1,428,0,FIN
3,1,B,MW,11:00:00,9/1/2004,Art,1,8,8,7,8,6,4,1,3.691959,1,428,0,FIN
4,1,B,MW,12:30:00,9/1/2005,SocialStudy,1,8,8,7,8,6,4,1,3.691959,1,428,0,FIN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9992,1000,C,MW,15:30:00,9/1/2008,Physics,1000,8,6,6,6,4,16,9,3.690760,0,552,1,FIN
9993,1000,C,MW,14:00:00,9/1/2008,Math,1000,8,6,6,6,4,16,9,3.690760,0,552,1,FIN
9994,1000,C,TTH,12:30:00,9/1/2008,ForeignLanguage,1000,8,6,6,6,4,16,9,3.690760,0,552,1,FIN
9995,1000,B,TU,19:00:00,9/1/2009,Music,1000,8,6,6,6,4,16,9,3.690760,0,552,1,FIN


In [11]:
# Making a list of all the dates and studentid's that have taken subjects in last 2 semester of their major. 
# Because we cannot predict the graduation based on last 2 semester's performance.

In [12]:
map={}
from datetime import datetime
for i in range(0,1000):
    list1=[]
    for j in range(fp.shape[0]):
        if fp.iloc[j,0]==i:
            list1.append(fp.iloc[j,4])
    list1=list(set(list1))
    list1.sort(key=lambda date: datetime.strptime(date, "%m/%d/%Y"))
    list1=list1[-2:]
    map[i]=list1    

In [13]:
map

{0: [],
 1: ['9/1/2006', '3/1/2007'],
 2: ['9/1/2006', '3/1/2007'],
 3: ['3/1/2005', '3/1/2006'],
 4: ['3/1/2007', '9/1/2007'],
 5: ['9/1/2008', '9/1/2009'],
 6: ['9/1/2007', '3/1/2008'],
 7: ['9/1/2005', '3/1/2006'],
 8: ['9/1/2007', '9/1/2008'],
 9: ['9/1/2005', '3/1/2006'],
 10: ['3/1/2011', '9/1/2011'],
 11: ['3/1/2007', '9/1/2007'],
 12: ['9/1/2007', '9/1/2008'],
 13: ['3/1/2007', '9/1/2007'],
 14: ['9/1/2005', '9/1/2006'],
 15: ['3/1/2005', '9/1/2005'],
 16: ['3/1/2011', '9/1/2011'],
 17: ['9/1/2011', '3/1/2012'],
 18: ['9/1/2006', '9/1/2008'],
 19: ['3/1/2006', '9/1/2006'],
 20: ['3/1/2006', '9/1/2006'],
 21: ['9/1/2011', '3/1/2012'],
 22: ['3/1/2007', '9/1/2007'],
 23: ['9/1/2005', '9/1/2006'],
 24: ['9/1/2009', '3/1/2010'],
 25: ['3/1/2005', '9/1/2005'],
 26: ['9/1/2005', '3/1/2006'],
 27: ['9/1/2007', '9/1/2008'],
 28: ['3/1/2006', '9/1/2006'],
 29: ['9/1/2005', '3/1/2006'],
 30: ['3/1/2006', '3/1/2007'],
 31: ['3/1/2006', '9/1/2006'],
 32: ['3/1/2005', '9/1/2005'],
 33: ['3/

In [14]:
# # Making another list of all the row numbers of dataset which needs to be deleted from dataframe
# because it contains last 2 semester's performance. 

In [15]:
indexList=[]
for index,row in fp.iterrows():
    if row["StudentID"]==1000:
        break
    dates = map[row["StudentID"]]
#     print(dates)
#     print(str(row["ClassDate"]) in dates)
#     print(index,row["StudentID"], row["ClassDate"])
    if str(row["ClassDate"]) in dates:
        indexList.append(index)
indexList

[8,
 9,
 17,
 18,
 19,
 25,
 26,
 27,
 28,
 29,
 37,
 38,
 39,
 47,
 48,
 49,
 57,
 58,
 59,
 65,
 66,
 67,
 68,
 69,
 76,
 77,
 78,
 79,
 87,
 88,
 89,
 98,
 99,
 106,
 107,
 108,
 109,
 118,
 119,
 128,
 129,
 134,
 135,
 136,
 137,
 138,
 139,
 147,
 148,
 149,
 157,
 158,
 159,
 167,
 168,
 169,
 177,
 178,
 179,
 187,
 188,
 189,
 198,
 199,
 207,
 208,
 209,
 217,
 218,
 219,
 226,
 227,
 228,
 229,
 238,
 239,
 247,
 248,
 249,
 256,
 257,
 258,
 259,
 266,
 267,
 268,
 269,
 277,
 278,
 279,
 287,
 288,
 289,
 297,
 298,
 299,
 307,
 308,
 309,
 316,
 317,
 318,
 319,
 326,
 327,
 328,
 329,
 337,
 338,
 339,
 347,
 348,
 349,
 357,
 358,
 359,
 366,
 367,
 368,
 369,
 377,
 378,
 379,
 386,
 387,
 388,
 389,
 398,
 399,
 407,
 408,
 409,
 418,
 419,
 425,
 426,
 427,
 428,
 429,
 436,
 437,
 438,
 439,
 447,
 448,
 449,
 458,
 459,
 467,
 468,
 469,
 475,
 476,
 477,
 478,
 479,
 488,
 489,
 495,
 496,
 497,
 498,
 499,
 507,
 508,
 509,
 514,
 515,
 516,
 517,
 518,
 519,
 52

list1

In [16]:
# Dropping all the rows from the dataframe.
# Got the final dataset on which we have data except the of last 2 semester data of individual students.

In [17]:
fp.shape
fp = fp.drop(indexList)
fp

,StudentID,GRADE,ClassWeekDay,ClassMeetTime,ClassDate,ClassTitle,ID,HS_ENGLISH,HS_MATH,HS_HISTORY,HS_LAB_SCIENCES,HS_FOREIGN_LANGUAGE,HS_ART,HS_ELECTIVES,HS_GPA,sex,WRITINGScore,GraduateONTime,Major
0,1,A,MW,15:30:00,9/1/2004,Math,1,8,8,7,8,6,4,1,3.691959,1,428,0,FIN
1,1,A,M,19:00:00,9/1/2004,English,1,8,8,7,8,6,4,1,3.691959,1,428,0,FIN
2,1,C,W,19:00:00,9/1/2004,Physics,1,8,8,7,8,6,4,1,3.691959,1,428,0,FIN
3,1,B,MW,11:00:00,9/1/2004,Art,1,8,8,7,8,6,4,1,3.691959,1,428,0,FIN
4,1,B,MW,12:30:00,9/1/2005,SocialStudy,1,8,8,7,8,6,4,1,3.691959,1,428,0,FIN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9992,1000,C,MW,15:30:00,9/1/2008,Physics,1000,8,6,6,6,4,16,9,3.690760,0,552,1,FIN
9993,1000,C,MW,14:00:00,9/1/2008,Math,1000,8,6,6,6,4,16,9,3.690760,0,552,1,FIN
9994,1000,C,TTH,12:30:00,9/1/2008,ForeignLanguage,1000,8,6,6,6,4,16,9,3.690760,0,552,1,FIN
9995,1000,B,TU,19:00:00,9/1/2009,Music,1000,8,6,6,6,4,16,9,3.690760,0,552,1,FIN


In [18]:
fp = fp.drop(['ClassMeetTime','ClassWeekDay','ID'], axis=1)
fp

,StudentID,GRADE,ClassDate,ClassTitle,HS_ENGLISH,HS_MATH,HS_HISTORY,HS_LAB_SCIENCES,HS_FOREIGN_LANGUAGE,HS_ART,HS_ELECTIVES,HS_GPA,sex,WRITINGScore,GraduateONTime,Major
0,1,A,9/1/2004,Math,8,8,7,8,6,4,1,3.691959,1,428,0,FIN
1,1,A,9/1/2004,English,8,8,7,8,6,4,1,3.691959,1,428,0,FIN
2,1,C,9/1/2004,Physics,8,8,7,8,6,4,1,3.691959,1,428,0,FIN
3,1,B,9/1/2004,Art,8,8,7,8,6,4,1,3.691959,1,428,0,FIN
4,1,B,9/1/2005,SocialStudy,8,8,7,8,6,4,1,3.691959,1,428,0,FIN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9992,1000,C,9/1/2008,Physics,8,6,6,6,4,16,9,3.690760,0,552,1,FIN
9993,1000,C,9/1/2008,Math,8,6,6,6,4,16,9,3.690760,0,552,1,FIN
9994,1000,C,9/1/2008,ForeignLanguage,8,6,6,6,4,16,9,3.690760,0,552,1,FIN
9995,1000,B,9/1/2009,Music,8,6,6,6,4,16,9,3.690760,0,552,1,FIN


In [19]:
fp.dtypes

StudentID                int32
GRADE                   object
ClassDate               object
ClassTitle              object
HS_ENGLISH               int64
HS_MATH                  int64
HS_HISTORY               int64
HS_LAB_SCIENCES          int64
HS_FOREIGN_LANGUAGE      int64
HS_ART                   int64
HS_ELECTIVES             int64
HS_GPA                 float64
sex                      int64
WRITINGScore             int64
GraduateONTime           int64
Major                   object
dtype: object

In [20]:
fp['GRADE'] = fp['GRADE'].astype('category')
fp['ClassDate'] = fp['ClassDate'].astype('category')
fp['ClassTitle'] = fp['ClassTitle'].astype('category')
fp['Major'] = fp['Major'].astype('category')

In [21]:
fp.dtypes

StudentID                 int32
GRADE                  category
ClassDate              category
ClassTitle             category
HS_ENGLISH                int64
HS_MATH                   int64
HS_HISTORY                int64
HS_LAB_SCIENCES           int64
HS_FOREIGN_LANGUAGE       int64
HS_ART                    int64
HS_ELECTIVES              int64
HS_GPA                  float64
sex                       int64
WRITINGScore              int64
GraduateONTime            int64
Major                  category
dtype: object

In [22]:
fp

,StudentID,GRADE,ClassDate,ClassTitle,HS_ENGLISH,HS_MATH,HS_HISTORY,HS_LAB_SCIENCES,HS_FOREIGN_LANGUAGE,HS_ART,HS_ELECTIVES,HS_GPA,sex,WRITINGScore,GraduateONTime,Major
0,1,A,9/1/2004,Math,8,8,7,8,6,4,1,3.691959,1,428,0,FIN
1,1,A,9/1/2004,English,8,8,7,8,6,4,1,3.691959,1,428,0,FIN
2,1,C,9/1/2004,Physics,8,8,7,8,6,4,1,3.691959,1,428,0,FIN
3,1,B,9/1/2004,Art,8,8,7,8,6,4,1,3.691959,1,428,0,FIN
4,1,B,9/1/2005,SocialStudy,8,8,7,8,6,4,1,3.691959,1,428,0,FIN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9992,1000,C,9/1/2008,Physics,8,6,6,6,4,16,9,3.690760,0,552,1,FIN
9993,1000,C,9/1/2008,Math,8,6,6,6,4,16,9,3.690760,0,552,1,FIN
9994,1000,C,9/1/2008,ForeignLanguage,8,6,6,6,4,16,9,3.690760,0,552,1,FIN
9995,1000,B,9/1/2009,Music,8,6,6,6,4,16,9,3.690760,0,552,1,FIN


In [24]:
le = preprocessing.LabelEncoder()
cols = ['GRADE','ClassDate','ClassTitle','Major']
fp[cols] = fp[cols].apply(le.fit_transform)

In [25]:
fp

,StudentID,GRADE,ClassDate,ClassTitle,HS_ENGLISH,HS_MATH,HS_HISTORY,HS_LAB_SCIENCES,HS_FOREIGN_LANGUAGE,HS_ART,HS_ELECTIVES,HS_GPA,sex,WRITINGScore,GraduateONTime,Major
0,1,0,7,4,8,8,7,8,6,4,1,3.691959,1,428,0,1
1,1,0,7,2,8,8,7,8,6,4,1,3.691959,1,428,0,1
2,1,2,7,7,8,8,7,8,6,4,1,3.691959,1,428,0,1
3,1,1,7,0,8,8,7,8,6,4,1,3.691959,1,428,0,1
4,1,1,8,8,8,8,7,8,6,4,1,3.691959,1,428,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9992,1000,2,11,7,8,6,6,6,4,16,9,3.690760,0,552,1,1
9993,1000,2,11,4,8,6,6,6,4,16,9,3.690760,0,552,1,1
9994,1000,2,11,3,8,6,6,6,4,16,9,3.690760,0,552,1,1
9995,1000,1,12,5,8,6,6,6,4,16,9,3.690760,0,552,1,1


In [26]:
target = fp['GraduateONTime']
predictors = fp.drop(['GraduateONTime'], axis=1)
X_train, X_test, y_train, y_test = train_test_split(predictors, target, test_size = 0.3, random_state = 0)


In [40]:
from sklearn.linear_model import LogisticRegression
classifier = LogisticRegression(random_state = 0)
classifier.fit(X_train, y_train)

C:\Users\User\anaconda3\envs\cloudoffis\lib\site-packages\sklearn\linear_model\_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,


LogisticRegression(random_state=0)

In [41]:
y_pred = classifier.predict(X_test)

In [42]:
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_pred)
print(cm)

[[1451    7]
 [ 527    6]]


In [43]:
print(metrics.classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           0       0.73      1.00      0.84      1458
           1       0.46      0.01      0.02       533

    accuracy                           0.73      1991
   macro avg       0.60      0.50      0.43      1991
weighted avg       0.66      0.73      0.62      1991



In [44]:
# Training the Random Forest Classification model on the Training set
from sklearn.ensemble import RandomForestClassifier
classifier = RandomForestClassifier(n_estimators = 10, criterion = 'entropy', random_state = 0)
classifier.fit(X_train, y_train)
y_pred = classifier.predict(X_test)



In [45]:
print(metrics.classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           0       0.99      1.00      0.99      1458
           1       1.00      0.96      0.98       533

    accuracy                           0.99      1991
   macro avg       0.99      0.98      0.99      1991
weighted avg       0.99      0.99      0.99      1991



In [46]:
# Training the K-NN model on the Training set
from sklearn.neighbors import KNeighborsClassifier
classifier = KNeighborsClassifier(n_neighbors = 5, metric = 'minkowski', p = 2)
classifier.fit(X_train, y_train)
y_pred = classifier.predict(X_test)


In [47]:
print(metrics.classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           0       0.96      0.97      0.97      1458
           1       0.93      0.89      0.91       533

    accuracy                           0.95      1991
   macro avg       0.94      0.93      0.94      1991
weighted avg       0.95      0.95      0.95      1991



In [1]:
! jupyter nbconvert --to html IS_670_Group_Project_2_Question_4.ipynb

[NbConvertApp] Converting notebook IS_670_Group_Project_2_Question_4.ipynb to html
[NbConvertApp] Writing 710575 bytes to IS_670_Group_Project_2_Question_4.html
